In [1]:
import polars as pl
import os

from tqdm import tqdm
import eikon as ek

ek.set_app_key("fd36c20b01ed4ad6b3248805ae5031749349e3c9")

In [ ]:
df, err = ek.get_data(
    instruments=["AAPL.OQ", "AMZN.OQ"],
    fields=["TR.REVENUE", "TR.F.TOTREVENUE"],
    parameters={
        "Scale": 6,
        "SDate": 0,
        "EDate": -25,
        "FRQ": "FY",
        "Curn": "Native"
    },  
    field_name=True
)
df

In [3]:
pl.read_parquet("assets\Eikon\data.parquet")

Instrument,TR.HEADQUARTERSCOUNTRY,TR.HEADQUARTERSCOUNTRY.DATE,TR.EMISSIONSTARGETTYPE,TR.EMISSIONSTARGETTYPE.DATE,TR.UPSTREAMSCOPE3EMISSIONSWASTEGENERATEDINOPERATIONS,TR.UPSTREAMSCOPE3EMISSIONSWASTEGENERATEDINOPERATIONS.DATE,TR.TARGETSEMISSIONS,TR.TARGETSEMISSIONS.DATE,TR.WASTEREDUCTIONINITIATIVES,TR.WASTEREDUCTIONINITIATIVES.DATE,TR.ENERGYUSETOTAL,TR.ENERGYUSETOTAL.DATE,TR.F.TOTREVENUE,TR.F.TOTREVENUE.DATE,TR.REVENUE,TR.REVENUE.DATE,TR.F.TOTLIAB,TR.F.TOTLIAB.DATE,TR.WASTETOTAL,TR.WASTETOTAL.DATE,TR.RENEWABLEENERGYPURCHASED,TR.RENEWABLEENERGYPURCHASED.DATE,TR.F.COGSUNCLASSIF,TR.F.COGSUNCLASSIF.DATE,TR.F.LOANSRCVBLTOT,TR.F.LOANSRCVBLTOT.DATE,TR.ENVIRONMENTALCONTROVERSIESCOUNT,TR.ENVIRONMENTALCONTROVERSIESCOUNT.DATE,TR.TOTALRENEWABLEENERGYTOENERGYUSEINMILLION,TR.TOTALRENEWABLEENERGYTOENERGYUSEINMILLION.DATE,TR.COMPANYNAME,TR.COMPANYNAME.DATE,TR.F.NONRECURINCEXPNTOT,TR.F.NONRECURINCEXPNTOT.DATE,TR.ENVIRONMENTALEXPENDITURES,TR.ENVIRONMENTALEXPENDITURES.DATE,…,TR.F.INCTAX.DATE,TR.VOCEMISSIONS,TR.VOCEMISSIONS.DATE,TR.ENVIRONMENTALMATERIALSSOURCING,TR.ENVIRONMENTALMATERIALSSOURCING.DATE,TR.GICSINDUSTRY,TR.GICSINDUSTRY.DATE,TR.ENERGYPRODUCEDDIRECT,TR.ENERGYPRODUCEDDIRECT.DATE,TR.RECENTENVIRONMENTALCONTROVERSIES,TR.RECENTENVIRONMENTALCONTROVERSIES.DATE,TR.F.EPSDILEXCLEXORDITEMSCOMTOT,TR.F.EPSDILEXCLEXORDITEMSCOMTOT.DATE,TR.F.TOTSHHOLDEQ,TR.F.TOTSHHOLDEQ.DATE,TR.CARBONOFFSETSCREDITS,TR.CARBONOFFSETSCREDITS.DATE,TR.WASTERECYCLEDTOTOTALWASTE,TR.WASTERECYCLEDTOTOTALWASTE.DATE,TR.UPSTREAMSCOPE3EMPLOYEECOMMUTING,TR.UPSTREAMSCOPE3EMPLOYEECOMMUTING.DATE,TR.F.INVNTTOT,TR.F.INVNTTOT.DATE,TR.GICSINDUSTRYCODE,TR.GICSINDUSTRYCODE.DATE,TR.SICINDUSTRYCODE,TR.SICINDUSTRYCODE.DATE,TR.F.COMEQPARENTSHHOLD,TR.F.COMEQPARENTSHHOLD.DATE,TR.WATERRECYCLED,TR.WATERRECYCLED.DATE,TR.F.COMSHROUTSTOT,TR.F.COMSHROUTSTOT.DATE,TR.F.INCBEFTAX,TR.F.INCBEFTAX.DATE,TR.RESOURCEREDUCTIONPOLICY,TR.RESOURCEREDUCTIONPOLICY.DATE
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""AAPL.OQ""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""AMZN.OQ""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [2]:
from source.refinitiv import EikonHelper

eikon_helper = EikonHelper()

Cell limit: 200000
Instrument limit: 20
Tickers retrieved: 2/62191
Batches remaining: 3110


In [ ]:
eikon_helper.extract()